In [3]:
import matplotlib.pyplot as plt
import plotly
import numpy as np
import random
import hrr
import math
import time
from plotly.graph_objs import Scatter, Layout, Surface
plotly.offline.init_notebook_mode(connected=True)

In [4]:
def log_transform(error):
    return math.copysign(1.0,error)*math.log(math.fabs(error)+1,2)

In [5]:
def softmax(arr,t=1.0):
    w = np.array(arr)
    e = np.exp(w / t)
    dist = e / np.sum(e)
    return dist

In [6]:
def argmax(arr_2d,restrict):
    max_row = restrict[0]
    max_col = 0
    #max_value = arr_2d[0,0]
    max_value = arr_2d[restrict[0],0]
    for row in range(arr_2d.shape[0]):
        if row not in restrict:
            continue
        for col in range(arr_2d.shape[1]):
            if arr_2d[row,col] > max_value:
                max_value = arr_2d[row,col]
                max_row,max_col = row,col
    return list((max_row,max_col))

In [7]:
def argmax1(arr_3d,outer,inner):
    max_row = outer[0]
    max_col = inner[0]
    max_x = 0
    #max_value = arr_2d[0,0]
    max_value = arr_3d[outer[0],inner[0],0]
    for row in range(arr_3d.shape[0]):
        if row not in outer:
            continue
        for col in range(arr_3d.shape[1]):
            if col not in inner:
                continue
            for x in range(arr_3d.shape[2]):
                if arr_3d[row,col,x] > max_value:
                    max_value = arr_3d[row,col,x]
                    max_row,max_col,max_x = row,col,x
    return list((max_row,max_col,max_x))

In [52]:
def Delayed_Saccade(nepisodes,ntrials,lrate,gamma,td_lambda,temp):
    n = 4000
    n_cross_locations = 5
    n_dot_locations = 5
    n_gaze_locations = 5
    n_dot_colors = 2
    n_cross_colors = 2
    #n_saccades = 5 # actions
    
    # working memory chuncks
    n_wm_crosshair_location = 5
    n_wm_dot_location = 5
    n_wm_agent_fixation = 5
    
    # identity vector
    hrr_i = np.zeros(n)
    hrr_i[0] = 1
    
    # saccade hrr
    agent_gaze_location = hrr.hrrs(n,n_gaze_locations)
    agent_gaze_location = np.row_stack((agent_gaze_location,hrr_i))
    
    # crosshair location hrr
    crosshair_location = hrr.hrrs(n,n_cross_locations)
    crosshair_location = np.row_stack((crosshair_location,hrr_i))
    
    # dot location hrr
    dot_location = hrr.hrrs(n,n_dot_locations)
    dot_location = np.row_stack((dot_location,hrr_i))
    
    # color hrr
    dot_color = hrr.hrrs(n,n_dot_colors)
    dot_color = np.row_stack((dot_color,hrr_i))
    
    # crosshair hrr
    crosshair_color = hrr.hrrs(n,n_cross_colors)
    crosshair_color = np.row_stack((crosshair_color,hrr_i))
    
    # dot color working memory hrr
    crosshair_color_wm = hrr.hrrs(n,n_cross_colors)
    crosshair_color_wm = np.row_stack((crosshair_color_wm,hrr_i))
    
    # agent fixation working memory hrr
    agent_fixation_wm = hrr.hrrs(n,n_wm_agent_fixation)
    agent_fixation_wm = np.row_stack((agent_fixation_wm,hrr_i))
    
    # dot location working memory hrr
    dot_location_wm = hrr.hrrs(n,n_wm_dot_location)
    dot_location_wm = np.row_stack((dot_location_wm,hrr_i))
    
    dot_color_wm = hrr.hrrs(n,n_dot_colors)
    dot_color_wm = np.row_stack((dot_color_wm,hrr_i))
    # crosshair location working memory hrr
    crosshair_location_wm = hrr.hrrs(n,n_wm_crosshair_location)
    crosshair_location_wm = np.row_stack((crosshair_location_wm,hrr_i))
    #print(dot_location[-1])
    #tick1 = time.time()
    #print('starting precompute ...')
    # precompute hrrs
    
    #tick2 = time.time()
    #print("time spent precomputing:",(tick2-tick1),"seconds")
    #print(representation.shape)
    # weight vector and bias
    W = hrr.hrr(n)
    bias = 1
    
    # epsilon for e-soft policy
    epsilon = .1
    
    # temperature for softmax
    t = temp
    
    # eligibility trace
    eligibility = np.zeros(n)
    center = 4
    nothing = 5
    correct_response_count = 0
    count = 0
    BLUE = 0
    PURPLE = 1
    RED = 0
    GREEN = 1 
    for episode in range(nepisodes):
        # 0 is blue 1 is purple
        cross_color = random.randrange(2)
        # 0 is red 1 is green
        target_color = random.randrange(2)
        if (episode+1) % 5 == 0:
                epsilon = 0.01
        for trial in range(ntrials):
            
            count += 1
            r = 0
            flag_count = 0 # used to determine if agent completed all stages correct
            eligibility = np.zeros(n)

            ######### nothing on the screen #############
            representation = hrr.convolve(agent_gaze_location,hrr.convolve(crosshair_location[-1],dot_location[-1]))

            values = np.dot(representation,W) + bias
            sm_prob = softmax(values,t)
            #print(sm_prob.shape)
            fixation = np.argmax(sm_prob)

            if random.random() < epsilon:
                fixation = random.randrange(n_gaze_locations)

            value = values[fixation]
            #error = (r+gamma*value) - pvalue
            #error = r - value
            #W += lrate*log_transform(error)*eligibility
            r = 0
            pvalue = value
            pfixation = fixation
            #eligibility = representation[fixation,:] + td_lambda*eligibility
            #e = hrr.convolve(agent_gaze_location[fixation],hrr.convolve(crosshair_location[-1],dot_location[-1]))
            eligibility = representation[fixation,:] + td_lambda*eligibility
            ###########################################


            ######## just the crosshair on the screen ###########
            representation = hrr.convolve(agent_gaze_location,hrr.convolve(crosshair_location[center],dot_location[-1]))
            representation = hrr.convolve(representation,crosshair_color[cross_color])
            representation = hrr.oconvolve(representation,crosshair_color_wm)
            representation = representation.reshape((n_cross_colors+1,n_gaze_locations+1,n))
            
            values = np.dot(representation,W) + bias
            #print(values)
            #print(values.shape)
            sm_prob = softmax(values,t)
            #print(sm_prob)
            # bootstraps agent towards looking at center crosshair
            possible_wm = np.unique(np.array([cross_color]))
            wm_action = argmax(sm_prob,possible_wm)
            wm_cross = wm_action[0]
            fixation = center#wm_action[1]

            #fixation = np.argmax(sm_prob)

            #print(wm_cross,wm_agent,action)
            #epsilon = .2
            if random.random() < epsilon:
                wm_cross = random.randrange(n_cross_colors)
                fixation = random.randrange(n_gaze_locations)

            value = values[wm_cross,fixation]
            error = (r+gamma*value) - pvalue
            W += lrate*log_transform(error)*eligibility

            pvalue = value
            pfixation = fixation

            eligibility = representation[wm_cross,fixation,:] + td_lambda*eligibility
            #e = hrr.convolve(agent_gaze_location[fixation],hrr.convolve(crosshair_location[-1],dot_location[-1]))

            if fixation == center:
                flag_count += 1
                r = 0

            #print(fixation,end=" ")
            ####################

            ########## crosshair and colored dot on the screen #######
            dot_loc = random.randint(0,3)
            representation = hrr.convolve(agent_gaze_location,hrr.convolve(crosshair_location[center],
                                        hrr.convolve(crosshair_color_wm[wm_cross],dot_location[dot_loc])))
            #wm = hrr.oconvolve(agent_fixation_wm,hrr.oconvolve(dot_location_wm,crosshair_location_wm))
            representation = hrr.oconvolve(representation,hrr.convolve(dot_color[target_color],
                                            hrr.oconvolve(dot_color_wm,dot_location_wm)))
            representation = representation.reshape((n_dot_colors+1,n_wm_dot_location+1,n_gaze_locations+1,n))
            #representation = representation.reshape((n_gaze_locations+1,n))
            #print(representation.shape)
            values = np.dot(representation,W) + bias
            sm_prob = softmax(values,t)
            #print(sm_prob.shape)
            
            possible_wm1 = np.unique(np.array([target_color]))
            possible_wm2 = np.unique(np.array([dot_loc]))
            wm_action = argmax1(sm_prob,possible_wm1,possible_wm2)

            wm_dot_color = wm_action[0]
            wm_dot_loc = wm_action[1]
            # bootstraps agent towards looking at center crosshair
            fixation = center#np.argmax(sm_prob)

            #fixation = wm_action[1]
            #print('wm:',wm_dot,dot_loc)
            #epsilon = .2
            if random.random() < epsilon:
                fixation = random.randrange(n_gaze_locations)
                wm_dot = random.randrange(n_gaze_locations-1)
            value = values[wm_dot_color,wm_dot_loc,fixation]
            #value = values[fixation]
            # update error and weights
            error = (r+gamma*value) - pvalue
            W += lrate*log_transform(error)*eligibility

            pvalue = value
            pfixation = fixation
            pwm = wm_dot_loc

            eligibility = representation[wm_dot_color,wm_dot_loc,fixation,:] + td_lambda*eligibility
            #eligibility = hrr.convolve(representation[fixation,:],dot_location_wm[wm_dot]) + td_lambda*eligibility
            if fixation == center:
                flag_count += 1
                r = 0

            #print(fixation,end=" ")
            ######################

            ####### just the crosshair on the screen ################

            representation = hrr.convolve(agent_gaze_location,hrr.convolve(crosshair_location[center],dot_location[-1]))
            representation = hrr.convolve(representation,hrr.convolve(dot_color_wm[wm_dot_color],dot_location_wm[wm_dot_loc]))

            representation = representation.reshape((n_gaze_locations+1,n))
            #print(representation.shape)
            values = np.dot(representation,W) + bias
            #print(values)
            #print(values.shape)
            sm_prob = softmax(values,t)
            #print(sm_prob)


            #print(wm_cross,wm_agent,action)
            #epsilon = .2
            if random.random() < epsilon:
                fixation = random.randrange(n_gaze_locations+1)

            value = values[fixation]
            error = (r+gamma*value) - pvalue
            W += lrate*log_transform(error)*eligibility

            pvalue = value
            pfixation = fixation
            pwm = wm_dot_loc
            #eligibility = representation[fixation,wm_agent,wm_crosshair,wm_dot,:] + td_lambda*eligibility
            #e = hrr.convolve(agent_gaze_location[fixation],hrr.convolve(crosshair_location[-1],dot_location[-1]))
            eligibility = representation[fixation,:] + td_lambda*eligibility
            if fixation == center:
                flag_count += 1
                r = 0

            #print(fixation,end=" ")
            ##############################

            ######## remove crosshair on the screen ################
            representation = hrr.convolve(agent_gaze_location,hrr.convolve(crosshair_location[-1],dot_location[-1]))
            representation = hrr.convolve(representation,dot_location_wm[wm_dot_loc])
            values = np.dot(representation,W) + bias
            sm_prob = softmax(values,t)
            #print(sm_prob.shape)
            fixation = np.argmax(sm_prob)

            # updated info

            if random.random() < epsilon:
                fixation = random.randrange(n_gaze_locations)
            
            if cross_color == BLUE:
                if target_color == RED:
                    if fixation != dot_loc:
                        flag_count += 1
                        r = 0
                elif target_color == GREEN:
                    if fixation == dot_loc:
                        flag_count += 1
                        r = 0
            elif cross_color == PURPLE:
                if target_color == GREEN:
                    if fixation != dot_loc:
                        flag_count += 1
                        r = 0
                elif target_color == RED:
                    if fixation == dot_loc:
                        flag_count += 1
                        r = 0
            value = values[fixation]
            error = (r+gamma*value) - pvalue
            #error = r - value
            W += lrate*log_transform(error)*eligibility

            pvalue = value
            pfixation = fixation
            eligibility = representation[fixation,:] + td_lambda*eligibility


            #############################

            #print('dot_loc:','(',dot_loc,')',end = ' ')
            #epsilon = .4
            '''
            if random.random() < epsilon:
                fixation = random.randrange(n_gaze_locations)

            if fixation == dot_loc:
                flag_count += 1 
                r = 0

            if flag_count == 4:
                r = 1
                print('correct')
                correct_response_count += 1 # used to track performance
            '''
            
            if flag_count == 4:
                r = 1
                correct_response_count += 1

            value = values[fixation]
            error = r - value
            W += lrate*log_transform(error)*eligibility

            #print(fixation)
        if episode % 10 == 0:
            performance = correct_response_count/count
            print('Episode:',episode)
            print('Trials:',count)
            print('Performance:',performance)
            count = 0
            correct_response_count = 0
    try:   
        performance = correct_response_count/count
        print('Episode:',episode)
        print('Trials:',count)
        print('Performance:',performance)
        count = 0
        correct_response_count = 0
    except ZeroDivisionError:
        print('Episode:',episode)
        print('Performance:',0)
    

In [54]:
Delayed_Saccade(100000,100,.1,.9,.9,.1)
# (num episodes, num trials, learning rate, discount factor(gamma), lambda, temperature)

Episode: 0
Trials: 100
Performance: 0.74
Episode: 10
Trials: 1000
Performance: 0.755
Episode: 20
Trials: 1000
Performance: 0.822
Episode: 30
Trials: 1000
Performance: 0.841
Episode: 40
Trials: 1000
Performance: 0.809
Episode: 50
Trials: 1000
Performance: 0.853
Episode: 60
Trials: 1000
Performance: 0.884
Episode: 70
Trials: 1000
Performance: 0.8
Episode: 80
Trials: 1000
Performance: 0.831
Episode: 90
Trials: 1000
Performance: 0.788
Episode: 100
Trials: 1000
Performance: 0.801
Episode: 110
Trials: 1000
Performance: 0.699
Episode: 120
Trials: 1000
Performance: 0.834
Episode: 130
Trials: 1000
Performance: 0.841
Episode: 140
Trials: 1000
Performance: 0.837
Episode: 150
Trials: 1000
Performance: 0.776
Episode: 160
Trials: 1000
Performance: 0.912
Episode: 170
Trials: 1000
Performance: 0.877
Episode: 180
Trials: 1000
Performance: 0.775
Episode: 190
Trials: 1000
Performance: 0.786
Episode: 200
Trials: 1000
Performance: 0.788
Episode: 210
Trials: 1000
Performance: 0.773
Episode: 220
Trials: 1000

KeyboardInterrupt: 

In [ ]:
l = [0,1,2,3,4]
l[0:1]
#np.argmax([0,1,2,3,4])

In [ ]:
gusget all 1160 LAB
gusrun all 1160 LAB
gus2pdf all 1160 LAB
----------
guscode all 1160 LAB
gusput all 1160 LAB